# Big-Data

This following paper has the porpouse to correlate some assumptions and verify some hypotheses about IMD's studants' grades:

    - Does distance effects grades? If so, which are the variables that make such effect;
    - Does it have any relationship between grades (materias escolares)?;

In [2]:
# Install necessary libraries to run this IPython Notebook
!pip install folium

## Treatment of data

In [3]:
import pandas as pd
import numpy as np
import folium
from folium.plugins import HeatMap

In [4]:
# Load spreadsheet
spreadsheet = pd.ExcelFile('imd_student_blind.xlsx')

# Load a sheet into a DataFrame by index
imd_student_blind_df = spreadsheet.parse(0)

In [5]:
# Print the head of the DataFrame df
imd_student_blind_df.head()

,a_ID,CEP,ano_ingresso,periodo_ingresso,status,ano_disciplina,periodo_disciplina,nota,disciplina_ID,status.disciplina
0,0,59015430,2014,1,CANCELADO,2014,2,2.6,0,Reprovado
1,0,59015430,2014,1,CANCELADO,2015,1,8.0,0,Aprovado
2,1,59073120,2014,1,CANCELADO,2014,2,0.1,0,Reprovado
3,2,59072580,2014,1,ATIVO,2014,2,6.1,0,Aprovado
4,3,59088150,2014,1,ATIVO,2014,1,3.0,0,Reprovado


In [30]:
# Print the columns' name
imd_student_blind_df.columns

Index(['a_ID', 'CEP', 'ano_ingresso', 'periodo_ingresso', 'status',
       'ano_disciplina', 'periodo_disciplina', 'nota', 'disciplina_ID',
       'status.disciplina'],
      dtype='object')

In [7]:
imd_student_blind_df.shape

(4842, 10)

In [55]:
id_cancel = imd_student_blind_df[imd_student_blind_df['status']=='CANCELADO'].groupby('ano_ingresso')['a_ID'].count()
id_cancel

ano_ingresso
2014    584
2015    329
2016    134
Name: a_ID, dtype: int64

In [39]:
from bokeh.charts import Bar, Histogram, output_notebook, show

p = Bar(id_by_year, values='a_ID', title="ID por ano cancelados", color='a_ID')

output_notebook()

show(p)

Loading BokehJS ...

This graph demonstrate which year has more students'resgistries that drop out faculty, so by that we can evaluate wich scholar matter caused more resgistration cancelled each year.

In [67]:
id_total_matter = imd_student_blind_df[imd_student_blind_df['status.disciplina'] == 'Reprovado'].groupby('disciplina_ID')['a_ID'].count()
id_total_matter

disciplina_ID
0    321
1    109
2    323
3    334
4    163
5    523
6    303
Name: a_ID, dtype: int64

In [72]:
p = Bar(id_total_matter, values='a_ID', title="ID's de todos os reprovados ao longo dos anos", color = 'a_ID')

output_notebook()

show(p)

Loading BokehJS ...

In [10]:
# Load geolocation extracted data
students_geolocation = pd.read_csv('students_geolocation.csv')

In [11]:
students_geolocation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900 entries, 0 to 899
Data columns (total 7 columns):
id               900 non-null int64
zipcode          900 non-null int64
total_entries    900 non-null int64
lat              736 non-null float64
lng              736 non-null float64
address          736 non-null object
distance         736 non-null float64
dtypes: float64(3), int64(3), object(1)
memory usage: 49.3+ KB


In [12]:
students_geolocation.head()

,id,zipcode,total_entries,lat,lng,address,distance
0,0,59015430,12,-5.816641,-35.200015,"Nova Descoberta, Natal - RN, 59015-430, Brazil",1.815783
1,1,59073120,6,-5.853337,-35.252804,"Planalto, Natal - RN, 59073-120, Brazil",5.749127
2,2,59072580,12,-5.832998,-35.242542,"Cidade Nova, Natal - RN, 59072-580, Brazil",4.114743
3,3,59088150,9,-5.872282,-35.206600,"Neópolis, Natal - RN, 59088-150, Brazil",4.439972
4,4,59064245,9,NaN,NaN,NaN,NaN


In [13]:
grouped_df = imd_student_blind_df.groupby('a_ID')

students_summary_df = grouped_df['nota'].agg({
    'grades_mean': 'mean',
    'grades_amount': 'count',
}).reset_index()

students_summary_df.rename(
    columns = {
        'a_ID' : 'id'
    },
    inplace = True
)

students_summary_df['distance'] = students_geolocation[students_geolocation['id'] == students_summary_df['id']]['distance']
students_summary_df['lat'] = students_geolocation[students_geolocation['id'] == students_summary_df['id']]['lat']
students_summary_df['lng'] = students_geolocation[students_geolocation['id'] == students_summary_df['id']]['lng']


In [14]:
students_summary_df.head()

,id,grades_mean,grades_amount,distance,lat,lng
0,0,2.916667,12,1.815783,-5.816641,-35.200015
1,1,3.216667,6,5.749127,-5.853337,-35.252804
2,2,4.116667,12,4.114743,-5.832998,-35.242542
3,3,5.588889,9,4.439972,-5.872282,-35.206600
4,4,3.422222,9,NaN,NaN,NaN


In [15]:
grades_x_distance = pd.DataFrame({
    'distance': students_summary_df['distance'],
    'grades_mean': students_summary_df['grades_mean']
})

# Drop NaN values
grades_x_distance.dropna(inplace = True)

# Get only entries that have less than 300km from distance
grades_x_distance = grades_x_distance[grades_x_distance['distance'] < 50]

In [16]:
grades_x_distance.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 596 entries, 0 to 899
Data columns (total 2 columns):
distance       596 non-null float64
grades_mean    596 non-null float64
dtypes: float64(2)
memory usage: 14.0 KB


## Graphics

In [17]:
# Import modules from Bokeh visualization library
#import matplotlib.pyplot as plt
#from bokeh.models import HoverTool
from bokeh.charts import Scatter
#from bokeh.layouts import row, gridplot
from bokeh.io import output_notebook, show
from bokeh.plotting import figure, ColumnDataSource

In [18]:
'''
data_groupby = ['ano_disciplina', 'periodo_disciplina', 'disciplina_ID']
nota = imd_student_blind_df.groupby(data_groupby)['nota']
nota.mean()
'''
# Gráfico
graph_lines = figure(plot_width=900, plot_height=300, title="Média de nota de cada disciplina, por período letivo")
colors = ['orange', 'blue', 'red', 'green', 'yellow', 'pink', 'purple', 'brown']
i = 0

# Array
ano_periodo_select = ['ano_disciplina', 'periodo_disciplina']

# Imprimir linhas de cada disciplina
for disciplina_ID in imd_student_blind_df['disciplina_ID'].unique():
    # Dados da disciplina
    data_disciplina = imd_student_blind_df[imd_student_blind_df['disciplina_ID'] == disciplina_ID]
    # Percorre o ano e o período
    ano_periodo = data_disciplina[ano_periodo_select].drop_duplicates()
    xline = []
    yline = []
    
    for ano in ano_periodo['ano_disciplina'].sort_values().unique():
        periodos_data = data_disciplina[data_disciplina['ano_disciplina'] == ano][['periodo_disciplina', 'nota']].sort_values('periodo_disciplina')
        for periodo in ano_periodo['periodo_disciplina'].sort_values().unique():
            # Se foi no primeiro ou segundo período
            if(periodo == 2):
                periodo_graph = 5
            else:
                periodo_graph = 0
            # X
            ap = "{0}.{1}".format(ano, periodo_graph)
            xline.append(float(ap))
            # Y
            media_periodo = periodos_data[periodos_data['periodo_disciplina'] == periodo]['nota'].mean()
            yline.append(float(media_periodo))
    # Data source para criar a linha
    line_disciplina_ds = ColumnDataSource(
        data=dict(
            x=xline,
            y=yline
        )
    )
    title = '{0}'.format(disciplina_ID)
    graph_lines.line('x', 'y', line_width=3, source=line_disciplina_ds, legend=title, color=colors[i])
    graph_lines.circle(xline, yline, color=colors[i], size=6, alpha=1.0)
    i += 1 # Avança no contador de cor
    
output_notebook()
show(graph_lines)

Loading BokehJS ...

In [19]:
p_grades_x_distance = Scatter(
        grades_x_distance,
        x='grades_mean',
        y='distance', 
        xlabel='Grades Mean',
        ylabel='Distance from IMD in kilometers',
        title='Grades Mean vs Distance from IMD',
        plot_width=800, plot_height=400
)

output_notebook()

show(p_grades_x_distance)

Loading BokehJS ...

In [20]:
# Create a coordinates list with the filtered data and show the heatmap

coordinates = []

for i in range(len(students_summary_df)):
    if all(~np.isnan([students_summary_df.ix[i,'lat'], students_summary_df.ix[i,'lng']])):
        coordinates.append([students_summary_df.ix[i,'lat'], students_summary_df.ix[i,'lng'], students_summary_df.ix[i,'grades_mean']])
        
f_map = folium.Map(
    location = [-5.791659, -35.28385],
    zoom_start = 11
)

HeatMap(coordinates).add_to(f_map)

f_map